In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers

In [3]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [4]:
from openpyxl import Workbook
from openpyxl import load_workbook

In [5]:
import pathlib
from datetime import datetime

In [6]:
import itertools
import os

In [7]:
(train_ds, validation_ds, test_ds), info = tfds.load(
    "colorectal_histology",
    split=["train[:85%]", "train[85%:95%]", "train[95%:]"],
    with_info=True,
    as_supervised=True,
    shuffle_files= True,
)

num_classes = info.features['label'].num_classes

In [8]:
size = (150, 150)
batch_size = 16

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=1)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=1)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=1)

In [10]:
%cd  functions

/home/reynier/py_projects/Deployments/functions


In [11]:
from Nets import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
from Simulation import save_obj, load_obj

In [12]:
model = GetNeuralNetworkModel('Xception',(150,150,3),8, quantization = False, aging_active=False)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 75, 75, 32)   896         ['input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 75, 75, 32)   0           ['conv2d[1][0]']                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 75, 75, 32)  128         ['lambda[1][0]']             

 lambda_15 (Lambda)             (None, 38, 38, 256)  0           ['activation_5[1][0]']           
                                                                                                  
 lambda_16 (Lambda)             (None, 38, 38, 256)  0           ['lambda_15[1][0]']              
                                                                                                  
 separable_conv2d_3 (SeparableC  (None, 38, 38, 256)  68096      ['lambda_16[1][0]']              
 onv2D)                                                                                           
                                                                                                  
 lambda_17 (Lambda)             (None, 38, 38, 256)  0           ['separable_conv2d_3[1][0]']     
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 38, 38, 256)  1024       ['lambda_17[1][0]']              
 rmalizati

                                                                                                  
 lambda_30 (Lambda)             (None, 10, 10, 728)  0           ['activation_10[1][0]']          
                                                                                                  
 lambda_31 (Lambda)             (None, 10, 10, 728)  0           ['lambda_30[1][0]']              
                                                                                                  
 separable_conv2d_8 (SeparableC  (None, 10, 10, 728)  537264     ['lambda_31[1][0]']              
 onv2D)                                                                                           
                                                                                                  
 lambda_32 (Lambda)             (None, 10, 10, 728)  0           ['separable_conv2d_8[1][0]']     
                                                                                                  
 batch_nor

                                                                                                  
 lambda_47 (Lambda)             (None, 10, 10, 728)  0           ['separable_conv2d_13[1][0]']    
                                                                                                  
 batch_normalization_15 (BatchN  (None, 10, 10, 728)  2912       ['lambda_47[1][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_16 (Activation)     (None, 10, 10, 728)  0           ['batch_normalization_15[1][0]'] 
                                                                                                  
 lambda_48 (Lambda)             (None, 10, 10, 728)  0           ['activation_16[1][0]']          
                                                                                                  
 lambda_49

                                                                                                  
 lambda_63 (Lambda)             (None, 10, 10, 728)  0           ['activation_21[1][0]']          
                                                                                                  
 lambda_64 (Lambda)             (None, 10, 10, 728)  0           ['lambda_63[1][0]']              
                                                                                                  
 separable_conv2d_19 (Separable  (None, 10, 10, 728)  537264     ['lambda_64[1][0]']              
 Conv2D)                                                                                          
                                                                                                  
 lambda_65 (Lambda)             (None, 10, 10, 728)  0           ['separable_conv2d_19[1][0]']    
                                                                                                  
 batch_nor

                                                                                                  
 lambda_80 (Lambda)             (None, 10, 10, 728)  0           ['separable_conv2d_24[1][0]']    
                                                                                                  
 batch_normalization_26 (BatchN  (None, 10, 10, 728)  2912       ['lambda_80[1][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_27 (Activation)     (None, 10, 10, 728)  0           ['batch_normalization_26[1][0]'] 
                                                                                                  
 lambda_81 (Lambda)             (None, 10, 10, 728)  0           ['activation_27[1][0]']          
                                                                                                  
 lambda_82

                                                                                                  
 lambda_96 (Lambda)             (None, 10, 10, 728)  0           ['activation_32[1][0]']          
                                                                                                  
 lambda_97 (Lambda)             (None, 10, 10, 728)  0           ['lambda_96[1][0]']              
                                                                                                  
 separable_conv2d_30 (Separable  (None, 10, 10, 728)  537264     ['lambda_97[1][0]']              
 Conv2D)                                                                                          
                                                                                                  
 lambda_98 (Lambda)             (None, 10, 10, 728)  0           ['separable_conv2d_30[1][0]']    
                                                                                                  
 batch_nor

In [14]:
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [15]:
%cd  ../Xception/Weights/

/home/reynier/py_projects/Deployments/Xception/Weights


In [16]:
model.load_weights('weights.data')

In [18]:
score = model.evaluate(train_ds, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

266/266 [==============================] - 17s 63ms/step - loss: 0.1886 - accuracy: 0.9313
Test loss: 0.18857821822166443
Test accuracy: 0.9312941431999207


In [19]:
for index,layer in enumerate(model.layers):
    print(index,layer.name)

0 input_2
1 conv2d
2 lambda
3 batch_normalization
4 activation
5 lambda_1
6 lambda_2
7 conv2d_1
8 lambda_3
9 batch_normalization_1
10 activation_1
11 lambda_4
12 lambda_5
13 activation_2
14 lambda_6
15 lambda_7
16 separable_conv2d
17 lambda_8
18 batch_normalization_2
19 activation_3
20 lambda_9
21 lambda_10
22 separable_conv2d_1
23 lambda_11
24 batch_normalization_3
25 max_pooling2d
26 conv2d_2
27 add
28 activation_4
29 lambda_12
30 lambda_13
31 separable_conv2d_2
32 lambda_14
33 batch_normalization_4
34 activation_5
35 lambda_15
36 lambda_16
37 separable_conv2d_3
38 lambda_17
39 batch_normalization_5
40 max_pooling2d_1
41 conv2d_3
42 add_1
43 activation_6
44 lambda_18
45 lambda_19
46 separable_conv2d_4
47 lambda_20
48 batch_normalization_6
49 activation_7
50 lambda_21
51 lambda_22
52 separable_conv2d_5
53 lambda_23
54 batch_normalization_7
55 max_pooling2d_2
56 conv2d_4
57 add_2
58 activation_8
59 lambda_24
60 lambda_25
61 separable_conv2d_6
62 lambda_26
63 batch_normalization_8
64 ac

In [22]:
num_address  = 753048     

Indices      = [1,7,16,22,25,26,31,37,40,41,46,52,55,56,61,67,73,80,86,92,99,105,
                111,118,124,130,137,143,149,156,162,168,175,181,187,194,200,206,213,
                219,222,223,228,234,237,242]

samples      = 150

Data         = GetReadAndWrites(model,Indices,num_address,samples,CNN_gating=False)
stats        = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
Baseline_Acceses   = pd.DataFrame(stats).reset_index(drop=False)

procesados:  25
procesados:  50
procesados:  75
procesados:  100
procesados:  125
procesados:  150


In [2]:
Indices      = [1,7,16,22,25,26,31,37,40,41,46,52,55,56,61,67,73,80,86,92,99,105,
                111,118,124,130,137,143,149,156,162,168,175,181,187,194,200,206,213,
                219,222,223,228,234,237,242]
len(Indices)

46

In [ ]:
%cd /home/reynier/py_projects/Deployments/Xception/Weights
df = QuantizationEffect('Xception',train_ds,'weights.data',(150,150,3),8,batch_size)

/home/reynier/py_projects/Deployments/Xception/Weights
Activation fraction part
0  bits results:  acc:  0.12423529475927353 loss:  20.979576110839844


/home/reynier/py_projects/Deployments/functions/Stats.py:209: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Experiment':['Activation fraction part'],'bits':[bit],'acc':[acc],'loss':[loss]}))


1  bits results:  acc:  0.27882352471351624 loss:  13.555887222290039


In [22]:
Accs = []
Loss = []
Volt = []

vol=54
for i in range(7):
    %cd '../../Data/byte_x/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../Xception/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('Xception', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 7, act_int_size = 8, wgt_frac_size = 14, wgt_int_size = 12,
                                            batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    Loss.append(loss)
    Volt.append(vol-1)    
    print(acc)
    
Acc_Net=pd.DataFrame(Accs)
Loss_Net=pd.DataFrame(Loss)
Volt_Net=pd.DataFrame(Volt)

buf_cero = pd.concat([Acc_Net, Loss_Net, Volt_Net], axis=1, join='outer')
buf_cero.columns =['Acc','Loss','Volt']
buf_cero.to_excel('Xception_fichero_fichero_Flip-patch.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 9439
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 252ms/step - loss: 2.0802 - accuracy: 0.6120
0.6119999885559082
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 3191
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 244ms/step - loss: 1.4031 - accuracy: 0.7880
0.7879999876022339
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 1126
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 240ms/step - loss: 1.0078 - accuracy: 0.7960
0.7960000038146973
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 337
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 233ms/step - loss: 0.4271 - accuracy: 0.8080
0.8080000281333923
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 56
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 223ms/step - loss: 0.2544 - accuracy: 0.9080
0.9079999923706055
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 6
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 2s 93ms/step - loss: 0.2366 - accuracy: 0.9040
0.9039999842643738
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 2
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 2s 93ms/step - loss: 0.2368 - accuracy: 0.9040
0.9039999842643738
 completada:  18:25:38
 completada:  18:25:38


In [21]:
Accs = []
Loss = []
Volt = []

vol=54
for i in range(7):
    %cd '../../Data/ECC/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../Xception/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('Xception', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 7, act_int_size = 8, wgt_frac_size = 14, wgt_int_size = 12,
                                            batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    Loss.append(loss)
    Volt.append(vol-1)    
    print(acc)
    
Acc_Net=pd.DataFrame(Accs)
Loss_Net=pd.DataFrame(Loss)
Volt_Net=pd.DataFrame(Volt)

buf_cero = pd.concat([Acc_Net, Loss_Net, Volt_Net], axis=1, join='outer')
buf_cero.columns =['Acc','Loss','Volt']
buf_cero.to_excel('Xception_fichero_fichero_ECC.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 9439
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 257ms/step - loss: 124.3639 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 3191
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 247ms/step - loss: 94.9072 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 1126
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 240ms/step - loss: 73.2741 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 337
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 230ms/step - loss: 28.6036 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 56
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 222ms/step - loss: 0.2367 - accuracy: 0.9040
0.9039999842643738
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 6
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 2s 92ms/step - loss: 0.2367 - accuracy: 0.9040
0.9039999842643738
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 2
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 2s 92ms/step - loss: 0.2367 - accuracy: 0.9040
0.9039999842643738
 completada:  18:24:29
 completada:  18:24:29


In [20]:
Accs = []
Loss = []
Volt = []

vol=54
for i in range(7):
    %cd '../../Data/ECCx/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../Xception/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('Xception', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 7, act_int_size = 8, wgt_frac_size = 14, wgt_int_size = 12,
                                            batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    Loss.append(loss)
    Volt.append(vol-1)    
    print(acc)
    
Acc_Net=pd.DataFrame(Accs)
Loss_Net=pd.DataFrame(Loss)
Volt_Net=pd.DataFrame(Volt)

buf_cero = pd.concat([Acc_Net, Loss_Net, Volt_Net], axis=1, join='outer')
buf_cero.columns =['Acc','Loss','Volt']
buf_cero.to_excel('Xception_fichero_fichero_Iso-Area_ECC.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 9439
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 257ms/step - loss: 104.8675 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 3191
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 244ms/step - loss: 104.2655 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 1126
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 239ms/step - loss: 111.4700 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 337
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 234ms/step - loss: 85.2836 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 56
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 229ms/step - loss: 2.1638 - accuracy: 0.5560
0.5559999942779541
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 6
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 2s 92ms/step - loss: 0.2368 - accuracy: 0.9040
0.9039999842643738
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 2
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 2s 92ms/step - loss: 0.2367 - accuracy: 0.9040
0.9039999842643738
 completada:  18:23:20
 completada:  18:23:20


In [18]:
Accs = []
Loss = []
Volt = []

vol=54
for i in range(7):
    %cd '../../Data/mask_volteada/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../Xception/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('Xception', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 7, act_int_size = 8, wgt_frac_size = 14, wgt_int_size = 12,
                                            batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    Loss.append(loss)
    Volt.append(vol-1)    
    print(acc)
    
Acc_Net=pd.DataFrame(Accs)
Loss_Net=pd.DataFrame(Loss)
Volt_Net=pd.DataFrame(Volt)

buf_cero = pd.concat([Acc_Net, Loss_Net, Volt_Net], axis=1, join='outer')
buf_cero.columns =['Acc','Loss','Volt']
buf_cero.to_excel('Xception_fichero_fichero_mask_volteada.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 9439
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 253ms/step - loss: 9.0995 - accuracy: 0.1840
0.18400000035762787
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 3191
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 248ms/step - loss: 7.3138 - accuracy: 0.2800
0.2800000011920929
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 1126
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 237ms/step - loss: 1.0078 - accuracy: 0.7960
0.7960000038146973
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 337
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 233ms/step - loss: 0.4546 - accuracy: 0.8040
0.8040000200271606
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 56
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 228ms/step - loss: 0.2544 - accuracy: 0.9080
0.9079999923706055
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 6
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 2s 93ms/step - loss: 0.2366 - accuracy: 0.9040
0.9039999842643738
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 2
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 2s 92ms/step - loss: 0.2368 - accuracy: 0.9040
0.9039999842643738
 completada:  18:21:01
 completada:  18:21:01


In [17]:
Accs = []
Loss = []
Volt = []

vol=54
for i in range(7):
    %cd '../../Data/base_line/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../Xception/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('Xception', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 7, act_int_size = 8, wgt_frac_size = 14, wgt_int_size = 12,
                                            batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    Loss.append(loss)
    Volt.append(vol-1)    
    print(acc)
    
Acc_Net=pd.DataFrame(Accs)
Loss_Net=pd.DataFrame(Loss)
Volt_Net=pd.DataFrame(Volt)

buf_cero = pd.concat([Acc_Net, Loss_Net, Volt_Net], axis=1, join='outer')
buf_cero.columns =['Acc','Loss','Volt']
buf_cero.to_excel('Xception_fichero_fichero_base_line.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/base_line/VC_707
tamaño de locs 9439
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 256ms/step - loss: 126.5510 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/base_line/VC_707
tamaño de locs 3191
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 244ms/step - loss: 111.8057 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/base_line/VC_707
tamaño de locs 1126
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 240ms/step - loss: 106.6739 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/base_line/VC_707
tamaño de locs 337
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 231ms/step - loss: 88.3205 - accuracy: 0.1320
0.13199999928474426
/home/reynier/py_projects/Deployments/Data/base_line/VC_707
tamaño de locs 56
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 4s 225ms/step - loss: 11.5842 - accuracy: 0.1680
0.1679999977350235
/home/reynier/py_projects/Deployments/Data/base_line/VC_707
tamaño de locs 6
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 2s 93ms/step - loss: 0.2368 - accuracy: 0.9040
0.9039999842643738
/home/reynier/py_projects/Deployments/Data/base_line/VC_707
tamaño de locs 2
/home/reynier/py_projects/Deployments/Xception/Weights


16/16 [==============================] - 2s 92ms/step - loss: 0.2367 - accuracy: 0.9040
0.9039999842643738
 completada:  18:19:03
 completada:  18:19:03
